In [8]:
import os
import sys
import operator
import time
import numpy as np
import contractions
from sklearn.model_selection import train_test_split


import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')  # needed by word_tokenize
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')
# list of words found to still rank very high (10k+) after a running with just stopwords and examining vocab.txt
other_words = ['lines', 'subject', 'would', 'organization']

# Assuming this file is put under the same parent directoray as the data directory, and the data directory is named "20news-train"
root_path = "./20news-train"
# The maximum size of the final vocabulary. It's a hyper-parameter. You can change it to see what value gives the best performance.
MAX_VOCAB_SIZE = 40000

start_time = time.time()
vocab_full = {}
n_doc = 0
# Only keep the data dictionaries and ignore possible system files like .DS_Store
folders = [os.path.join(root_path, name) for name in os.listdir(root_path) if os.path.isdir(os.path.join(root_path, name))]
for folder in folders:
    for filename in os.listdir(folder):
        file = os.path.join(folder, filename)
        n_doc += 1
        # print(file)
        with open(file, 'r', encoding='utf8', errors='ignore') as f:
            for line in f:
                # split contractions into two words
                line = contractions.fix(line)
                tokens = word_tokenize(line)
                # force everything to lower case and remove non-alphabetic characters
                tokens = [token.lower() for token in tokens if token.isalpha()]
                for token in tokens:
                    # remove stop words, other words (above) and single characters
                    if (token not in stop_words) and (token not in other_words) and (len(token) > 1):
                        vocab_full[token] = vocab_full.get(token, 0) + 1
print(f'{n_doc} documents in total with a total vocab size of {len(vocab_full)}')
vocab_sorted = sorted(vocab_full.items(), key=operator.itemgetter(1), reverse=True)
vocab_truncated = vocab_sorted[:MAX_VOCAB_SIZE]
# Save the vocabulary to file for visual inspection and possible analysis
with open('vocab1.txt', 'w') as f:
    for vocab, freq in vocab_truncated:
        f.write(f'{vocab}\t{freq}\n')
# The final vocabulary is a dict mapping each token to its id. frequency information is not needed anymore.
vocab = dict([(token, id) for id, (token, _) in enumerate(vocab_truncated)])
# Since we have truncated the vocabulary, we will encounter many tokens that are not in the vocabulary. We will map all of them to the same 'UNK' token (a common practice in text processing), so we append it to the end of the vocabulary.
vocab['UNK'] = MAX_VOCAB_SIZE
vocab_size = len(vocab)
unk_id = MAX_VOCAB_SIZE
elapsed_time = time.time() - start_time
print(f'Vocabulary construction took {elapsed_time} seconds')

[nltk_data] Downloading package punkt to C:\Users\Matthew
[nltk_data]     Ray\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Matthew
[nltk_data]     Ray\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


FileNotFoundError: [WinError 3] The system cannot find the path specified: './20news-train'

In [2]:
# Since we have truncated the vocabulary, it's now reasonable to hold the entire feature matrix in memory (it takes about 3.6GB on a 64-bit machine). If memory is an issue, you could make the vocabulary even smaller or use sparse matrix.
start_time = time.time()
features = np.zeros((n_doc, vocab_size), dtype=int)
print(f'The feature matrix takes {sys.getsizeof(features)} Bytes.')
# The class label of each document
labels = np.zeros(n_doc, dtype=int)
# The mapping from the name of each class label (i.e., the subdictionary name corresponding to a topic) to an integer ID
label2id = {}
label_id = 0
doc_id = 0
for folder in folders:
    label2id[folder] = label_id
    for filename in os.listdir(folder):
        labels[doc_id] = label_id
        file = os.path.join(folder, filename)
        with open(file, 'r', encoding='utf8', errors='ignore') as f:
            for line in f:
                tokens = word_tokenize(line)
                for token in tokens:
                    # if the current token is in the vocabulary, get its ID; otherwise, get the ID of the UNK token
                    token_id = vocab.get(token, unk_id)
                    features[doc_id, token_id] += 1
        doc_id += 1
    label_id += 1
elapsed_time = time.time() - start_time
print(f'Feature extraction took {elapsed_time} seconds')

The feature matrix takes 452605368 Bytes.
Feature extraction took 55.472546339035034 seconds


In [10]:
# k-nearest neighbor functions
from scipy.spatial.distance import euclidean
from scipy.stats import mode
from sklearn.model_selection import train_test_split
import numpy as np


neighbors = 8
training_rounds = 5


# Locate the most similar neighbors
def get_neighbor_labels(train_x, train_y, test_row):
	distances = list()
	# use i to mark the index of the train_row within train_x
	i = 0
	for train_row in train_x:
		dist = euclidean(test_row, train_row)
		distances.append((i, dist))
		i += 1
	distances.sort(key=lambda tup: tup[1])
	neighbor_labels = list()
	for i in range(neighbors):
		# get the index of the neighbors from the training data, then look up their label and add it
		location = distances[i][0]
		neighbor_labels.append(train_y[location])
	return neighbor_labels


def classify_one_row(train_x, train_y, test_row):
	nearest_labels = get_neighbor_labels(train_x, train_y, test_row)
	# ties are settled by selecting the smallest value
	mode_results = mode(nearest_labels)
	# if all values tie at 1, select the closest label
	if mode_results.count[0] == 1:
		result = nearest_labels[0]
	else:
		result = mode_results.mode[0]
	return result


def classify_test_data(train_x, train_y, test_x):
	predictions = list()
	for test_row in test_x:
		predictions.append(classify_one_row(train_x, train_y, test_row))
	return predictions


def accuracy(predicted_labels, test_y):
	correct = 0
	for i in range(len(predicted_labels)):
		if predicted_labels[i] == test_y[i]:
			correct += 1
	score = correct / float(len(test_y)) * 100
	print('{} correct predictions were made for a score of {}%'.format(correct, score))


def evaluate(features, labels):
	for _ in range(training_rounds):
		x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)
		predictions = classify_test_data(x_train, y_train, x_test)
		accuracy(predictions, y_test)

In [ ]:
# sample run of knn, can probably skip
evaluate(features, labels)

In [ ]:
# since the decision tree is a model that can be built, make a class so the tree can be on object built from node objects

class Node:
    def __init__(self, gini, num_samples, num_samples_per_class, predicted_class):
        self.gini = gini
        self.num_samples = num_samples
        self.num_samples_per_class = num_samples_per_class
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None

# based on https://github.com/scikit-learn/scikit-learn/blob/95d4f0841/sklearn/tree/_classes.py#L585
# and https://ysu1989.github.io/courses/sp20/cse5243/Classification-BasicConcepts.pdf
class tree_classifier:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def fit(self, features, labels):
        self.n_classes_ = len(set(labels))  # labels are assumed to go from 0 to n-1
        self.n_features_ = features.shape[1]
        self.tree_ = self._grow_tree(features, labels)

    def predict(self, features):
        return [self._predict(inputs) for inputs in features]

    def _gini(self, labels):
        return 1.0 - sum((np.sum(labels == labels.size) / labels.size) ** 2 for c in range(self.n_classes_))

    def _best_split(self, features, labels):
        if labels.size <= 1:
            return None, None

        # Count of each class in the current node.
        num_parent = [np.sum(labels == c) for c in range(self.n_classes_)]

        # Gini of current node with no split
        best_gini = 1.0 - sum((n / labels.size) ** 2 for n in num_parent)
        best_index, best_threshold = None, None

        # Loop through all features to determine best split
        for index in range(self.n_features_):
            # Sort data along selected feature.  threshold contains the count for the given feature(word).
            # Labels is the label of the document
            thresholds, labels = zip(*sorted(zip(features[:, index], labels)))

            num_left = [0] * self.n_classes_
            num_right = num_parent.copy()
            for i in range(1, labels.size):  # try sliding threshold (27-34)
                c = labels[i - 1]
                num_left[c] += 1
                num_right[c] -= 1
                gini_left = 1.0 - sum(
                    (num_left[x] / i) ** 2 for x in range(self.n_classes_)
                )
                gini_right = 1.0 - sum(
                    (num_right[x] / (labels.size - i)) ** 2 for x in range(self.n_classes_)
                )

                # weighted average as on slide 21
                gini = (i * gini_left + (labels.size - i) * gini_right) / labels.size

                # The following condition is to make sure we don't try to split two
                # points with identical values for that feature, as it is impossible
                # (both have to end up on the same side of a split).
                if thresholds[i] == thresholds[i - 1]:
                    continue

                if gini < best_gini:
                    best_gini = gini
                    best_index = index
                    best_threshold = (thresholds[i] + thresholds[i - 1]) / 2  # midpoint

        return best_index, best_threshold

    def _grow_tree(self, features, labels, depth=0):
        # Population for each class in current node. The predicted class is the one with
        # largest population.
        num_samples_per_class = [np.sum(labels == i) for i in range(self.n_classes_)]
        predicted_class = np.argmax(num_samples_per_class)
        node = Node(
            gini=self._gini(labels), num_samples=labels.size, num_samples_per_class=num_samples_per_class, predicted_class=predicted_class,)

        # Split recursively until maximum depth is reached.
        if depth < self.max_depth:
            index, thr = self._best_split(features, labels)
            if index is not None:
                indices_left = features[:, index] < thr
                X_left, y_left = features[indices_left], labels[indices_left]
                X_right, y_right = features[~indices_left], labels[~indices_left]
                node.feature_index = index
                node.threshold = thr
                node.left = self._grow_tree(X_left, y_left, depth + 1)
                node.right = self._grow_tree(X_right, y_right, depth + 1)
        return node

    def _predict(self, inputs):
        node = self.tree_
        while node.left:
            if inputs[node.feature_index] < node.threshold:
                node = node.left
            else:
                node = node.right
        return node.predicted_class

In [ ]:
# decision tree test, can also probably be skipped
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)
tree = tree_classifier(5)
tree.fit(x_train, y_train)
predictions = list()
for test_row in x_test:
    predictions.append(tree.predict(test_row.reshape(1, -1)))
    
correct = 0
i = 0
for label in predictions:
    if label[0] == y_test[i]:
        correct += 1
score = correct / float(len(y_test)) * 100
print('{} correct predictions were made for a score of {}%'.format(correct, score))

In [7]:
# preprocesses and feature selection of test set, copied from above with root path changed

import os
import sys
import operator
import time
import numpy as np
import contractions
from sklearn.model_selection import train_test_split


import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')  # needed by word_tokenize
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')
# list of words found to still rank very high (10k+) after a running with just stopwords and examining vocab.txt
other_words = ['lines', 'subject', 'would', 'organization']

# Assuming this file is put under the same parent directoray as the data directory, and the data directory is named "20news-train"
root_path = "./20news-test"
# The maximum size of the final vocabulary. It's a hyper-parameter. You can change it to see what value gives the best performance.
MAX_VOCAB_SIZE = 5000

start_time = time.time()
vocab_full_test = {}
n_doc_test = 0
# Only keep the data dictionaries and ignore possible system files like .DS_Store
folders = [os.path.join(root_path, name) for name in os.listdir(root_path) if os.path.isdir(os.path.join(root_path, name))]
for folder in folders:
    for filename in os.listdir(folder):
        file = os.path.join(folder, filename)
        n_doc_test += 1
        # print(file)
        with open(file, 'r', encoding='utf8', errors='ignore') as f:
            for line in f:
                # split contractions into two words
                line = contractions.fix(line)
                tokens = word_tokenize(line)
                # force everything to lower case and remove non-alphabetic characters
                tokens = [token.lower() for token in tokens if token.isalpha()]
                for token in tokens:
                    # remove stop words, other words (above) and single characters
                    if (token not in stop_words) and (token not in other_words) and (len(token) > 1):
                        vocab_full_test[token] = vocab_full_test.get(token, 0) + 1
print(f'{n_doc_test} documents in total with a total vocab size of {len(vocab_full_test)}')
vocab_sorted_test = sorted(vocab_full_test.items(), key=operator.itemgetter(1), reverse=True)
vocab_truncated_test = vocab_sorted_test[:MAX_VOCAB_SIZE]
# Save the vocabulary to file for visual inspection and possible analysis
with open('vocab1.txt', 'w') as f:
    for vocab_test, freq in vocab_truncated_test:
        f.write(f'{vocab_test}\t{freq}\n')
# The final vocabulary is a dict mapping each token to its id. frequency information is not needed anymore.
vocab_test = dict([(token, id) for id, (token, _) in enumerate(vocab_truncated_test)])
# Since we have truncated the vocabulary, we will encounter many tokens that are not in the vocabulary. We will map all of them to the same 'UNK' token (a common practice in text processing), so we append it to the end of the vocabulary.
vocab_test['UNK'] = MAX_VOCAB_SIZE
vocab_size_test = len(vocab_test)
unk_id = MAX_VOCAB_SIZE
elapsed_time = time.time() - start_time
print(f'Vocabulary test construction took {elapsed_time} seconds')

[nltk_data] Downloading package punkt to C:\Users\Matthew
[nltk_data]     Ray\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Matthew
[nltk_data]     Ray\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


11314 documents in total with a total vocab size of 72370
Vocabulary test construction took 53.990609645843506 seconds


In [11]:
# Since we have truncated the vocabulary, it's now reasonable to hold the entire feature matrix in memory (it takes about 3.6GB on a 64-bit machine). If memory is an issue, you could make the vocabulary even smaller or use sparse matrix.
start_time = time.time()
features_test = np.zeros((n_doc_test, vocab_size_test), dtype=int)
print(f'The feature matrix takes {sys.getsizeof(features)} Bytes.')
# The class label of each document
labels_test = np.zeros(n_doc_test, dtype=int)
# The mapping from the name of each class label (i.e., the subdictionary name corresponding to a topic) to an integer ID
label2id = {}
label_id = 0
doc_id = 0
for folder in folders:
    label2id[folder] = label_id
    for filename in os.listdir(folder):
        labels_test[doc_id] = label_id
        file = os.path.join(folder, filename)
        with open(file, 'r', encoding='utf8', errors='ignore') as f:
            for line in f:
                tokens = word_tokenize(line)
                for token in tokens:
                    # if the current token is in the vocabulary, get its ID; otherwise, get the ID of the UNK token
                    token_id = vocab_test.get(token, unk_id)
                    features_test[doc_id, token_id] += 1
        doc_id += 1
    label_id += 1
elapsed_time = time.time() - start_time
print(f'Feature test extraction took {elapsed_time} seconds')

NameError: name 'n_doc_test' is not defined

In [ ]:
# this time use the whole database, rather than the splitting done in evaluate.  the original features are labels are
# like the train x/y and the test features and labels the test x/y
test_predicitons = classify_test_data(features, labels, features_test)
accuracy(test_predicitions, labels_test)